<a href="https://colab.research.google.com/github/malinphy/q_17/blob/main/pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scann -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 60.4 MB/s eta 0:00:00


In [3]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense
from keras.models import Model
import matplotlib.pyplot as plt
import scann
plt.style.use('ggplot')
# import openai
from sklearn.decomposition import PCA

In [4]:

vector_dir = 'drive/MyDrive/BBC_content_vectors.npy'
title_dir = 'drive/MyDrive/transfer/title_vectors_gpt.npy'
embedding_vectors = np.load(vector_dir)
title_gpt_vectors = np.squeeze(np.load(title_dir))
print((embedding_vectors).shape)
print(title_gpt_vectors.shape)

(15468, 1536)
(15468, 1536)


In [5]:
pca_ = PCA(n_components=256, random_state = 42, svd_solver = 'arpack')
embedding_vectors_pca = pca_.fit_transform(embedding_vectors)
title_vectors_pca = pca_.transform(title_gpt_vectors)

In [6]:
searcher = scann.scann_ops_pybind.builder(embedding_vectors, 25, "dot_product").tree(
    num_leaves=1500, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [7]:
searcher_pca = scann.scann_ops_pybind.builder(embedding_vectors_pca, 25, "dot_product").tree(
    num_leaves=1500, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [8]:
raw_idx = []
reduced_idx = []
for i in range(20):
    raw_idx.append(searcher.search(title_gpt_vectors[i])[0][:10])
    reduced_idx.append(searcher_pca.search(title_vectors_pca[i])[0][:10])

In [9]:
def hit_rate(actual, predicted):
    return np.intersect1d(actual, predicted)

In [10]:
total = 0
for i in range(len(raw_idx)):
    intersection = hit_rate(raw_idx[i], reduced_idx[i])
    total = total + len(intersection)
    print(i, len(intersection), intersection)

0 7 [    0   484  1517  2415  8305  8902 12584]
1 7 [    1  1965  2569  2832  4164  8938 10916]
2 5 [   2 1259 1909 5616 6728]
3 7 [   3  104 3029 3997 4993 5652 7606]
4 1 [4]
5 4 [   5 3244 3422 7469]
6 4 [   6 1461 3712 6484]
7 4 [    7  8741 13910 15327]
8 3 [    8  1380 14349]
9 6 [    9  1953  4117  7077 13584 13967]
10 5 [   10  1318  4809 13282 13514]
11 5 [   11  2509 11156 13365 15356]
12 7 [   12  2544  5574 13008 13763 14011 15178]
13 9 [   13  1456  5227  8505  9320 10151 11788 14003 15180]
14 7 [   14   333  6102  6130  9779 15091 15429]
15 5 [   15  5096 12604 13205 14615]
16 2 [  16 7364]
17 6 [   17  1840  3557  4013 12210 13387]
18 5 [   18  4235  9398  9630 11318]
19 4 [   19  8059 10255 15234]


In [ ]:
7.0
7.0
5.0
7.0
1.0
4.0
4.0
4.0
3.0
6.0
5.0
5.0
7.0
9.0
7.0
5.0
2.0
6.0
5.0
4.0

In [32]:
reduced_idx = np.array(reduced_idx, dtype = 'int32')
raw_idx = np.array(raw_idx, dtype = 'int32')

In [42]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # if not actual:
    #     return 0.0
    # print(num_hits)
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])
    # return np.average([apk(a,p,k) for a,p in zip(actual, predicted)])


mapk(raw_idx, reduced_idx, k=3)    

0.8777777777777777

In [ ]:
ap = []
# loop through and calculate AP for each query q
for q in range(Q):
    ap_num = 0
    # loop through k values
    for x in range(k):
        # calculate precision@k
        act_set = set(actual[q])                                                                                                                                   
        pred_set = set(predicted[:x+1])
        precision_at_k = len(act_set & pred_set) / (x+1)
        # calculate rel_k values
        if predicted[x] in actual[q]:
            rel_k = 1
        else:
            rel_k = 0
        # calculate numerator value for ap
        ap_num += precision_at_k * rel_k
    # now we calculate the AP value as the average of AP
    # numerator values
    ap_q = ap_num / len(actual[q])
    print(f"AP@{k}_{q+1} = {round(ap_q,2)}")
    ap.append(ap_q)

# now we take the mean of all ap values to get mAP
map_at_k = sum(ap) / Q

# generate results
print(f"mAP@{k} = {round(map_at_k, 2)}")